In [290]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from numpy import random
import pprint

#reference to coursera course

#Create Data and Prepocessing


In [126]:
#gen 3-d data
n = int(input())
f = open('INP.txt',"w")
f.write("{} \n".format(n))

for i in range(0,n):
    l_gen = []
    for j in range(0,3):
        g = round(random.uniform(0,100),3)
        l_gen.append(g)
    if i != n - 1:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))
        f.write('\n')
    else:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))

f.close()

1000


In [139]:
#preprocess-data => convert to 3d vector for local hashing
points = []
with open('INP.txt','r',encoding='UTF-8') as f:
     n = int(f.readline())
     for i in range(n):
         line = f.readline()
         lst = re.split("\s+",line)
         if len(lst) >= 4:
            lst = lst[:-1]
         point = [round(float(x),3) - 50.000 for x in lst]
         point = np.array(point).reshape(1,3)
         points.append(point)
points = np.array(points)

#Helper Function


In [209]:
def cosine_similarity(a,b):
    dot = np.sum(a * b)
    cosin = dot/(np.linalg.norm(a,2) * np.linalg.norm(b,2))

    return cosin 

def euclide_distance(a,b):

    dist = 0.0
    for i in range(len(a)):
        dist += np.sum(pow((a[i] - b[i])/50.0,2))

    dist = np.sqrt(dist)
    return dist

#LSH (Locality Sensitive Hashing)


In [195]:
N_VECS = len(points)
N_DIMS = points[0].shape[1]
print(f"Data has {N_VECS} elements and each point has {N_DIMS} dimensions")

Data has 1000 elements and each point has 3 dimensions


In [214]:
#we want to create 100/4 = 25 buckets
#each plane create 2 parts => n planes create 2^n parts
#=> 2^n = 25 => n ~= 5

print("Enter the number of vectors you want to store in a bucket:")
n_vecs = int(input())

#compute N_PLANES:
N_PLANES = int(np.ceil(np.log2(N_VECS/n_vecs)))
#impore the search
N_UNIVERSES = 10

Enter the number of vectors you want to store in a bucket:
10


In [215]:
#random planes
random.seed(0)
planes_l = [np.random.normal(size = (N_DIMS,N_PLANES)) for _ in range(N_UNIVERSES)]

In [216]:
def hash_v(v,planes):
    dot_product = np.dot(v,planes) # (1,3) * (3,4)
    sign_of_dot_product = np.sign(dot_product)
    h = sign_of_dot_product >= 0
    #convert 2D to 1D array
    h = np.squeeze(h)
    hash_value = 0
    n_planes = planes.shape[1]
    for i in range(n_planes):
        hash_value += pow(2,i) * h[i]
    hash_value = int(hash_value)

    return hash_value

#test hash function

In [217]:
def create_hash_table(vecs,planes):
    num_of_planes = planes.shape[1]
    num_of_buckets = pow(2,num_of_planes)

    #dictionary hash_table store value
    hash_table = {i : [] for i in range(num_of_buckets)}

    #dictionary id_table store id of value
    id_table = {i : [] for i in range(num_of_buckets)}

    for i,v in enumerate(vecs):

        h = hash_v(v,planes)

        #store v in "h" buckets
        hash_table[h].append(v)

        #store id of v in "h" id_buckets
        id_table[h].append(i)

    return hash_table,id_table

#test_hash_table

In [218]:
hash_tables = []
id_tables = []

for id in range(N_UNIVERSES):
    print("set up hash-table with version: ",id)
    planes = planes_l[id]
    hash_table,id_table = create_hash_table(points,planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)    

set up hash-table with version:  0
set up hash-table with version:  1
set up hash-table with version:  2
set up hash-table with version:  3
set up hash-table with version:  4
set up hash-table with version:  5
set up hash-table with version:  6
set up hash-table with version:  7
set up hash-table with version:  8
set up hash-table with version:  9


In [219]:
def knn(v, v_neighbor_l, k=1):
    similarity_l = []

    for ele in v_neighbor_l:
        cos_similarity = cosine_similarity(v,ele)

        similarity_l.append(cos_similarity)
        
    sorted_idxs = np.argsort(similarity_l)
    
    k_idx = sorted_idxs[-k:]

    return k_idx

def knn_euclide(v, v_neighbor_l, k = 1):
    similarity_l = []

    for ele in v_neighbor_l:
        dist = euclide_distance(v,ele)

        similarity_l.append(dist)
    
    sorted_idxs = np.argsort(similarity_l)

    k_idx = sorted_idxs[-k:]

    return k_idx

In [221]:
#not flip method
def find_k_nearest_neighbor_universe(id_query,point_query,planes_l,k = 1,num_universes = N_UNIVERSES):
    
    points_neighbor_l = list()
    idxs_neighbor_l = list()
    idxs_neighbor_set = set() #check faster if point already exist in set

    for uni_id in range(num_universes):
        planes = planes_l[uni_id]

        #get hash_value of point_query to find neareast neighbor of this
        hash_value = hash_v(point_query,planes)

        hash_table = hash_tables[uni_id]
        id_table = id_tables[uni_id]

        #get subset of hash_table to lookup
        new_ids_table_consider = id_table[hash_value]
        new_points_table_consider = hash_table[hash_value]
        
        #if id_query exist in hash_table
        if id_query in new_ids_table_consider:
           new_ids_table_consider.remove(id_query)

        for i,new_id in enumerate(new_ids_table_consider):
            if new_id not in idxs_neighbor_set:
               point_i = new_points_table_consider[i]
              
               #add list to consider
               points_neighbor_l.append(point_i)
               idxs_neighbor_l.append(new_id)

               #add set to faster checking
               idxs_neighbor_set.add(new_id)

    print("Number of points to consider: ",len(points_neighbor_l))    
    points_neighbor_l_arr = np.array(points_neighbor_l)
    
    k_nearest_neighbor_idx = knn(point_query,points_neighbor_l_arr,k)
    k_nearest_neighbor_idx_l = [idxs_neighbor_l[idx] for idx in k_nearest_neighbor_idx]

    k_nearest_neighbor_idx_point = [points[idx] for idx in k_nearest_neighbor_idx_l]

    return k_nearest_neighbor_idx_l,k_nearest_neighbor_idx_point

In [285]:
#flip method
np.random.seed(1)
plane = np.random.normal(size = (N_DIMS,N_PLANES))

def flipped_one_bits(hash):
    bits = bin(hash).format(N_PLANES)[2:]

    hashs = list()
    
    for i in range(len(bits)):
        bits_temp = list(bits)  
        if bits[i] == '0':
           bits_temp[i] = '1'
        else:
           bits_temp[i] = '0'
        bits_temp = ''.join(bits_temp)
        hashs.append(int(bits_temp,2))
    return hashs

def flipped_two_bits(hash):
    bits = bin(hash).format(N_PLANES)[2:]

    hashs = list()

    for i in range(len(bits)):
        for j in range(i + 1,len(bits)):
            bits_temp = list(bits)
            
            bits_temp[i] = chr(1 - (int(bits[i]) - 48))
            bits_temp[j] = chr(1 - (int(bits[j]) - 48))

            bits_temp = ''.join(bits_temp)
            hashs.append(int(bits_temp,2))
    return hashs


def find_k_nearest_neighbor_flipped(id_query,point_query,plane,k = 3):
    points_neighbor_l = list()
    idxs_neighbor_l = list()
    idxs_neighbor_set = set()

    hash_tables,id_tables = create_hash_table(points,plane)
    hash_value = hash_v(point_query,plane)

    two_bits = flipped_two_bits(hash_value)
    one_bits = flipped_one_bits(hash_value)

    hashs = list()
    hashs.append(hash_value)

    hashs = hashs + one_bits + two_bits
    
    for hash in hashs:
        #subset of table
        hash_table_subset = hash_tables[hash]
        id_table_subset = id_tables[hash]
        
        if id_query in id_table_subset:
           id_table_subset.remove(id_query)

        for i,id in enumerate(id_table_subset):
            if id not in idxs_neighbor_set:
               
               point_i = hash_table_subset[i]

               points_neighbor_l.append(point_i)
               idxs_neighbor_l.append(id)

               idxs_neighbor_set.add(id)

    print("Number of points to consider: ",len(points_neighbor_l))    
    points_neighbor_l_arr = np.array(points_neighbor_l)
    
    k_nearest_neighbor_idx = knn(point_query,points_neighbor_l_arr,k)
    k_nearest_neighbor_idx_l = [idxs_neighbor_l[idx] for idx in k_nearest_neighbor_idx]

    k_nearest_neighbor_idx_point = [points[idx] for idx in k_nearest_neighbor_idx_l]

    return k_nearest_neighbor_idx_l,k_nearest_neighbor_idx_point     


In [284]:
two_bits = flipped_two_bits(100)
one_bits = flipped_one_bits(100)

[100, 36, 68, 116, 108, 96, 102, 101]
[100, 4, 52, 44, 32, 38, 37, 84, 76, 64, 70, 69, 124, 112, 118, 117, 104, 110, 109, 98, 97, 103]


In [330]:
#69.743 45.354 72.206
id_query = 0
point_query = points[0]

id_l,point_l = find_k_nearest_neighbor_flipped(id_query,point_query,plane,k = 10)

point_normal = [ele + 50.00 for ele in point_l]

k_l = knn(point_query,points,10)
knn_l = [points[ele] + 50.00 for ele in k_l]

def check(a,b):
    a = np.squeeze(a)
    b = np.squeeze(b)  
    for i in range(len(a)):
        if a[i] != b[i]:
           return False
    return True

def accuracy(predict_l,label_l):
    count = 0.0
    for i in range(len(predict_l)):
        for j in range(i + 1,len(label_l)):
            if check(predict_l[i],label_l[j]) is True:            
               count += 1
               break

    acc = (count * 1.0) / len(predict_l)
    return acc

accuracy(point_normal,knn_l)

Number of points to consider:  339


0.2